In [2]:
RR = RealField(1000)


def log2(x): return log(x,2)

def bits_to_bytes(x):
    byts = x/8
    KiB = 2^(10)
    MiB = 2^(20)
    GiB = 2^(30)
    TiB = 2^(40)
    if(byts < KiB):
        return "{} B".format(floor(byts))
    elif(byts < MiB):
        return "{} KiB".format(floor(byts/KiB))
    elif(byts < GiB):
        return "{} MiB".format(RR(byts/MiB).str(digits=3))
    elif(byts < TiB):
        return "{} GiB".format(RR(byts/GiB).str(digits=3))
    else:
        return "{} TiB".format(RR(byts/TiB).str(digits=3))

        

def proof_size(hash_out, ell, k, n, F):
    f_bits = (2*k*ell+1)*ceil(log2(F))
    hash_number = k + ell*( (n+1)*(n+2) - (n-k+1)*(n-k+2) )
    hash_bits = hash_out*hash_number
    
    total_bits = f_bits + hash_bits
    return(bits_to_bytes(total_bits))


def batch_proof_size(hash_out, ell, k, n, F, t):
    f_bits = (2*ell*(k+t)+1)*ceil(log2(F))
    hash_number = k + ell*( (n+1)*(n+2) - (n-k+1)*(n-k+2) ) + t*(1 + 2*ell*(n+1))
    hash_bits = hash_out * hash_number
    
    total_bits = f_bits + hash_bits
    return(bits_to_bytes(total_bits))




def calc_bound_provable(nu, lgQ_list, F_list, k_list, rho_list, kappa):
    for lgQ in lgQ_list:
        Q = 2^(lgQ) # number of RO queries we're allowing an adversary to make
        hash_out = kappa # hash output length
        print("nu =", nu)
        print("\\hhline{|t:==:t|}")
        out_head = "\\multicolumn{2}{||c||}{$\\nu = " + str(nu) + "$} "
        out_head = out_head + "& \\multicolumn{4}{c}{\\cref{cor:fs-fri}} \\\\\\hhline{||~~||----}\n"
        out_head = out_head + "\\multicolumn{2}{||c||}{$Q = 2^{" +str(lgQ) + "}$} "
        for rho in rho_list:
            out_head = out_head + "& $\\rho = " + str(rho) + "$ "
        out_head = out_head + "\\\\\\hhline{|b:==:b|*{4}{=|}}"
        print(out_head)
        for F in F_list:
            #print("\tField of size 2^{f}".format(f=log(F,2)))
            out = "{} ".format(log2(F))
            out2 = ""
            for k in k_list:
                #print("\tDomain of size 2^{n}".format(n=log(N,2)))
                out2 = out2 + "& {} ".format(k)
                for rho in rho_list:
                    delta = 0 # FRI proximity parameter to calculate
                    valid = True # check if computation below has valid constraints
                    m=0 # Johnson proximity parameter to compute
                    n=floor(log2((2^k)/rho)) # log2 size of the evaluation domain/codeword length
                    N = 2^n
                    
                    # do calculations for provable levels of security
                    m = floor( ((3*(rho^(3/2))*F) / (N^2 * Q * (2^(nu+1))))^(1/7) - (1/2) )
                    if(3 > m):
                        valid = False # constraint is violated
                    else:
                        delta = 1-sqrt(rho)*(1+(1/(2*m)))


                    if(valid):
                        #ell = ceil( log2( (1/(Q*2^(nu+1))) ) / log2(1-delta) )
                        #rbr = (((m+1/2)^7) * ((2^n)^2)) / ( 3*((rho)^(3/2))*F )
                        upper = 1/(Q*(2^(nu+1)))
                        ell = ceil( log2(upper) / log2(1-delta) )
                        #print(ell)
                        #print(ceil(log2(upper)/log2(1-delta)))
                        out2 = out2 + "& \\small({}, {}, {}) ".format(RR(delta).str(digits=3),ell,proof_size(hash_out, ell, k, n, F))
                    else:
                        out2 = out2 + "& \\cellcolor{lightgray} "

                out2 = out2 + "\\\\\n"
            print(out + out2[:-1] + "\\hhline{*{6}{-}}")
        out_last = "$\\log|\\bbF|$ & $k$ & \\multicolumn{4}{c}{$(\\delta, \\ell, |\\pi|)$} \\\\"
        print(out_last)
        print()
    return


## calculation for [BCI+20a] parameters
def calc_bound_conj_1(nu, lgQ_list, F_list, k_list, rho_list, kappa):
    for lgQ in lgQ_list:
        Q = 2^(lgQ) # number of RO queries we're allowing an adversary to make
        hash_out = kappa # hash output length
        print("nu =", nu)
        print("\\hhline{|t:==:t|}")
        out_head = "\\multicolumn{2}{||c||}{$\\nu = " + str(nu) + "$} "
        out_head = out_head + "& \\multicolumn{4}{c}{\\cref{conj:fri-1}} \\\\\\hhline{||~~||----}\n"
        out_head = out_head + "\\multicolumn{2}{||c||}{$Q = 2^{" +str(lgQ) + "}$} "
        for rho in rho_list:
            out_head = out_head + "& $\\rho = " + str(rho) + "$ "
        out_head = out_head + "\\\\\\hhline{|b:==:b|*{4}{=|}}"
        print(out_head)
        for F in F_list:
            #print("\tField of size 2^{f}".format(f=log(F,2)))
            out = "{} ".format(log2(F))
            out2 = ""
            for k in k_list:
                #print("\tDomain of size 2^{n}".format(n=log(N,2)))
                out2 = out2 + "& {} ".format(k)
                for rho in rho_list:
                    delta = 0 # FRI proximity parameter to calculate
                    valid = True # check if computation below has valid constraints
                    eta=0 
                    n=floor(log2((2^k)/rho)) # log2 size of the evaluation domain/codeword length
                    N = 2^n
                    
                    # do calculations for conjectured levels of security
                    eta = (N*Q*(2^(nu+1))) / (rho * F)
                    if(eta <= 0 or eta >= rho):
                        valid = False # constraint is violated
                    else:
                        delta = 1-rho-eta


                    if(valid):
                        upper = 1/(Q*(2^(nu+1)))
                        ell = ceil( log2(upper) / log2(1-delta) )
                        #print(ell)
                        #print(ceil(log2(upper)/log2(1-delta)))
                        out2 = out2 + "& \\small({}, {}, {}) ".format(RR(delta).str(digits=3),ell,proof_size(hash_out, ell, k, n, F))
                    else:
                        out2 = out2 + "& \\cellcolor{lightgray} "

                out2 = out2 + "\\\\\n"
            print(out + out2[:-1] + "\\hhline{*{6}{-}}")
        out_last = "$\\log|\\bbF|$ & $k$ & \\multicolumn{4}{c}{$(\\delta, \\ell, |\\pi|)$} \\\\"
        print(out_last)
        print()
    return


## calculation for ethSTARK conjectured parameters
def calc_bound_conj_2(nu, lgQ_list, F_list, k_list, rho_list, kappa):
    for lgQ in lgQ_list:
        hash_out = kappa # hash output length
        print("nu =", nu)
        print("\\hhline{|t:==:t|}")
        out_head = "\\multicolumn{2}{||c||}{$\\nu = " + str(nu) + "$} "
        out_head = out_head + "& \\multicolumn{4}{c}{\\cref{conj:fri-2}} \\\\\\hhline{||~~||----}\n"
        out_head = out_head + "\\multicolumn{2}{||c||}{$Q = 2^{" +str(lgQ) + "}$} "
        for rho in rho_list:
            out_head = out_head + "& $\\rho = " + str(rho) + "$ "
        out_head = out_head + "\\\\\\hhline{|b:==:b|*{4}{=|}}"
        print(out_head)
        for F in F_list:
            out = "{} ".format(log2(F))
            out2 = ""
            Q = 0
            lgQ_valid = floor(log2(F/2^(nu+1)))
            valid = True
            if(lgQ > lgQ_valid):
                valid = False
            else:
                Q = 2^(lgQ)

            for k in k_list:
                out2 = out2 + "& {} ".format(k)
                for rho in rho_list:
                    delta = 1-rho # FRI proximity parameter 
                    n=floor(log2((2^k)/rho)) # log2 size of the evaluation domain/codeword length
                    N = 2^n

                    if(valid):
                        ell = ceil( log2( (1/(Q*2^(nu+1))) ) / log2(1-delta) )
                        out2 = out2 + "& \\small({}, {}, {}) ".format(RR(delta).str(digits=3),ell,proof_size(hash_out, ell, k, n, F))
                    else:
                        out2 = out2 + "& \\cellcolor{lightgray} "

                out2 = out2 + "\\\\\n"
            print(out + out2[:-1] + "\\hhline{*{6}{-}}")
        out_last = "$\\log|\\bbF|$ & $k$ & \\multicolumn{4}{c}{$(\\delta, \\ell, |\\pi|)$} \\\\"
        print(out_last)
        print()
    return


#### PART 1 CALCULATIONS ####
# target field sizes
F_list = [2^i for i in [128, 192, 256]]

# log of message lengths
k_list = [i for i in [10, 15, 20, 25]] 

# target rates
rho_list = [1/2^i for i in [1, 2, 3, 4]] 

# target security levels
nu_list = [20, 40,60,80] 

# hash output length
kappa = 256 


lgQ_list = [60, 80]
## compute bounds for provable security and print tables
print("===== Provable Security Calculations =====")
for nu in nu_list:
    Q_upper = floor( sqrt( ((2^(kappa-nu-1))/3) - 1 )) # upper bound on number of RO queries
    lgQ_upper = floor(log2(Q_upper))
    print("$2^{",str(lgQ_upper),"} \\leq Q < 2^{", str(lgQ_upper+1), "}$") # print powers of 2 range of Q_upper
    #lgQ_list = [floor(lgQ_upper/10)*10-i for i in [30,20,10,0]]
    
    # calculate bounds and print tables
    calc_bound_provable(nu, lgQ_list, F_list, k_list, rho_list, kappa)

## compute bounds for conjectured security from [BCI+20a] and print tables
print("===== Conjectured Security Calculations 1 =====")
for nu in nu_list:
    Q_upper = floor( sqrt( ((2^(kappa-nu-1))/3) - 1 )) # upper bound on number of RO queries
    lgQ_upper = floor(log2(Q_upper))
    print("$2^{",str(lgQ_upper),"} \\leq Q < 2^{", str(lgQ_upper+1), "}$") # print powers of 2 range of Q_upper
    #lgQ_list = [floor(lgQ_upper/10)*10-i for i in [30,20,10,0]]
    
    # calculate bounds and print tables
    calc_bound_conj_1(nu, lgQ_list, F_list, k_list, rho_list, kappa)

## compute bounds for conjectured security from ethSTARK and print tables
print("===== Conjectured Security Calculations 2 =====")
for nu in nu_list:
    Q_upper = floor( sqrt( ((2^(kappa-nu-1))/3) - 1 )) # upper bound on number of RO queries
    lgQ_upper = floor(log2(Q_upper))
    print("$2^{",str(lgQ_upper),"} \\leq Q < 2^{", str(lgQ_upper+1), "}$") # print powers of 2 range of Q_upper
    
    calc_bound_conj_2(nu, lgQ_list, F_list, k_list, rho_list, kappa)



#### PART 2 CALCULATIONS ####

## In the following 3 functions, I use ell to switch modes:
##  - ell >= 1 indicates we are using a verifier query
##     bound that is given as a parameter by prior work.
##  - ell < 1 indicates that I want to calculate ell such
##     that the remaining constraints are satisfied.


def provable_calcs(F, rho, k, m, lgQ_list, kappa, ell): 
    delta = 1-sqrt(rho)*(1+1/(2*m))
    N = (2^k)/rho
    rbr = 0
    if(ell >= 1):
        rbr = max( ((m+1/2)^7 * N^2) / ( 3*(rho^(3/2))*F ), (1-delta)^ell )
    else:
        rbr = ((m+1/2)^7 * N^2) / ( 3*(rho^(3/2))*F )
    for i in lgQ_list:
        Q = 2^i
        nu1 = log2( 1 / (2*Q*rbr) ) 
        nu2 = log2( 2^(kappa-1) / (3*(Q^2+1)) )
        nu = floor(min(nu1,nu2))
        if(ell < 1):
            ell = ceil( log2( 1 / (Q*(2^(nu+1))) ) / log2(1-delta) )
        proof_len = proof_size(kappa, ell, k, log2(N), F)
        print("$Q=2^{",i,"}$,","({}, {}, {}, {})".format(nu, RR(delta).str(digits=5), ell, proof_len))
    return

def conj_calcs_1(F, rho, k, eta, c1, c2, lgQ_list, kappa, ell):
    delta = 1-rho-eta
    N = (2^k)/rho
    rbr = 0
    if(ell >= 1):
        rbr = max( (N^(c2)) / ( ((rho*eta)^(c1)) * F ), (1-delta)^ell)
    else:
        rbr = (N^(c2)) / ( ((rho*eta)^(c1)) * F )
    for i in lgQ_list:
        Q = 2^i
        nu1 = log2( 1 / (2*Q*rbr) ) 
        nu2 = log2( 2^(kappa-1) / (3*(Q^2+1)) )
        nu = floor(min(nu1,nu2))
        if(ell < 1):
            ell = ceil( log2( 1 / (Q*(2^(nu+1))) ) / log2(1-delta) )
        proof_len = proof_size(kappa, ell, k, log2(N), F)
        print("$Q=2^{",i,"}$,","({}, {}, {}, {})".format(nu, RR(delta).str(digits=5), ell, proof_len))
    return

def conj_calcs_2(F, rho, k, lgQ_list, kappa, ell):
    delta = 1-rho
    N = (2^k)/rho
    rbr = 0
    if(ell >= 1):
        rbr = max(1/F, (1-delta)^ell)
    else:
        rbr = 1/F
    for i in lgQ_list:
        Q = 2^i
        nu1 = log2( 1 / (2*Q*rbr) ) 
        nu2 = log2( 2^(kappa-1) / (3*(Q^2+1)) )
        nu = floor(min(nu1,nu2))
        if(ell < 1):
            ell = ceil( log2( 1 / (Q*(2^(nu+1))) ) / log2(1-delta) )
        proof_len = proof_size(kappa, ell, k, log2(N), F)
        print("$Q=2^{",i,"}$,","({}, {}, {}, {})".format(nu, RR(delta).str(digits=5), ell, proof_len))
    return




### set targets for number of RO queries ###
lgQ_list = [20*i for i in range(1,5)]
kappa = 256

### calculations for Ben-Sasson et al. [BCI+20a] Parameters ###
F = 2^(256)
rho = 1/16
k = 16
m = 2^(11)-1
ell = 65

## provable security calculations 
print()
print("=== Ben-Sasson et al. Parameters ===")
print("Provable security calculations")

provable_calcs(F, rho, k, m, lgQ_list, kappa, ell)
    
## [BCI+20a] conjectured security calculations
eta = rho/(2*m)
c1 = 1
c2 = 2

print()
print("Conjectured security calculations 1")

conj_calcs_1(F, rho, k, eta, c1, c2, lgQ_list, kappa, ell)
    
## ethstark conjectured security calculations

print()
print("Conjectured security calculations 2")

conj_calcs_2(F, rho, k, lgQ_list, kappa, ell)

    
    
    
    
### calculations for RISC Zero Parameters ###

print()
print("=== RISC Zero Parameters ===")

F = 2^(124)
rho = 1/4
k = 24
ell = 50

## provable security calculations 
print("Provable security calculations")

m=3
provable_calcs(F, rho, k, m, lgQ_list, kappa, ell)
    
## [BCI+20a] conjectured security calculations
eta = rho/(2*m)
c1 = 1
c2 = 2

print()
print("Conjectured security calculations 1")

conj_calcs_1(F, rho, k, eta, c1, c2, lgQ_list, kappa, ell)
    
## ethstark conjectured security calculations

print()
print("Conjectured security calculations 2")

conj_calcs_2(F, rho, k, lgQ_list, kappa, ell)
    

    
    
### calculations for Plonky2 Parameters, Part 1 ###
print()
print("=== Plonky2 Parameters 1 ===")
F = 2^(128)
rho = 1/2
k = 31
ell = 84

## provable security calculations 
print("Provable security calculations")

m=3
provable_calcs(F, rho, k, m, lgQ_list, kappa, ell)
    
## [BCI+20a] conjectured security calculations
eta = rho/6
c1 = 1
c2 = 2

print()
print("Conjectured security calculations 1")

conj_calcs_1(F, rho, k, eta, c1, c2, lgQ_list, kappa, ell)
    
## ethstark conjectured security calculations

print()
print("Conjectured security calculations 2")

conj_calcs_2(F, rho, k, lgQ_list, kappa, ell)


    
    
    

### calculations for Plonky2 Parameters, Part 2 ###
print()
print("=== Plonky2 Parameters 2 ===")

F = 2^(128)
rho = 1/8
k = 29
ell = 28

## provable security calculations 
print("Provable security calculations")
m=3
provable_calcs(F, rho, k, m, lgQ_list, kappa, ell)
    
## [BCI+20a] conjectured security calculations
eta = rho/6
c1 = 1
c2 = 2

print()
print("Conjectured security calculations 1")

conj_calcs_1(F, rho, k, eta, c1, c2, lgQ_list, kappa, ell)
    
## ethstark conjectured security calculations

print()
print("Conjectured security calculations 2")

conj_calcs_2(F, rho, k, lgQ_list, kappa, ell)

===== Provable Security Calculations =====
$2^{ 116 } \leq Q < 2^{ 117 }$
nu = 20
\hhline{|t:==:t|}
\multicolumn{2}{||c||}{$\nu = 20$} & \multicolumn{4}{c}{\cref{cor:fs-fri}} \\\hhline{||~~||----}
\multicolumn{2}{||c||}{$Q = 2^{60}$} & $\rho = 1/2$ & $\rho = 1/4$ & $\rho = 1/8$ & $\rho = 1/16$ \\\hhline{|b:==:b|*{4}{=|}}
128 & 10 & \small(0.261, 186, 930 KiB) & \small(0.464, 90, 506 KiB) & \small(0.611, 60, 375 KiB) & \small(0.708, 46, 316 KiB) \\
& 15 & \small(0.175, 292, 2.81 MiB) & \cellcolor{lightgray} & \cellcolor{lightgray} & \cellcolor{lightgray} \\
& 20 & \cellcolor{lightgray} & \cellcolor{lightgray} & \cellcolor{lightgray} & \cellcolor{lightgray} \\
& 25 & \cellcolor{lightgray} & \cellcolor{lightgray} & \cellcolor{lightgray} & \cellcolor{lightgray} \\\hhline{*{6}{-}}
192 & 10 & \small(0.293, 163, 840 KiB) & \small(0.500, 82, 474 KiB) & \small(0.646, 55, 352 KiB) & \small(0.750, 41, 288 KiB) \\
& 15 & \small(0.293, 163, 1.60 MiB) & \small(0.500, 82, 903 KiB) & \small(0.646, 55,

192 & 10 & \small(0.500, 82, 423 KiB) & \small(0.750, 41, 237 KiB) & \small(0.875, 28, 179 KiB) & \small(0.937, 21, 147 KiB) \\
& 15 & \small(0.500, 82, 826 KiB) & \small(0.750, 41, 452 KiB) & \small(0.875, 28, 335 KiB) & \small(0.937, 21, 271 KiB) \\
& 20 & \small(0.500, 82, 1.33 MiB) & \small(0.750, 41, 730 KiB) & \small(0.875, 28, 534 KiB) & \small(0.937, 21, 427 KiB) \\
& 25 & \small(0.500, 82, 1.97 MiB) & \small(0.750, 41, 1.05 MiB) & \small(0.875, 28, 777 KiB) & \small(0.937, 21, 616 KiB) \\\hhline{*{6}{-}}
256 & 10 & \small(0.500, 82, 435 KiB) & \small(0.750, 41, 243 KiB) & \small(0.875, 28, 184 KiB) & \small(0.937, 21, 151 KiB) \\
& 15 & \small(0.500, 82, 846 KiB) & \small(0.750, 41, 461 KiB) & \small(0.875, 28, 341 KiB) & \small(0.937, 21, 276 KiB) \\
& 20 & \small(0.500, 82, 1.35 MiB) & \small(0.750, 41, 743 KiB) & \small(0.875, 28, 543 KiB) & \small(0.937, 21, 433 KiB) \\
& 25 & \small(0.500, 82, 2.00 MiB) & \small(0.750, 41, 1.06 MiB) & \small(0.875, 28, 788 KiB) & \small(0

256 & 10 & \small(0.500, 142, 754 KiB) & \small(0.750, 71, 421 KiB) & \small(0.875, 48, 315 KiB) & \small(0.937, 36, 259 KiB) \\
& 15 & \small(0.500, 142, 1.43 MiB) & \small(0.750, 71, 799 KiB) & \small(0.875, 48, 585 KiB) & \small(0.937, 36, 473 KiB) \\
& 20 & \small(0.500, 142, 2.34 MiB) & \small(0.750, 71, 1.26 MiB) & \small(0.875, 48, 930 KiB) & \small(0.937, 36, 743 KiB) \\
& 25 & \small(0.500, 142, 3.47 MiB) & \small(0.750, 71, 1.84 MiB) & \small(0.875, 48, 1.32 MiB) & \small(0.937, 36, 1.04 MiB) \\\hhline{*{6}{-}}
$\log|\bbF|$ & $k$ & \multicolumn{4}{c}{$(\delta, \ell, |\pi|)$} \\

$2^{ 86 } \leq Q < 2^{ 87 }$
nu = 80
\hhline{|t:==:t|}
\multicolumn{2}{||c||}{$\nu = 80$} & \multicolumn{4}{c}{\cref{conj:fri-1}} \\\hhline{||~~||----}
\multicolumn{2}{||c||}{$Q = 2^{60}$} & $\rho = 1/2$ & $\rho = 1/4$ & $\rho = 1/8$ & $\rho = 1/16$ \\\hhline{|b:==:b|*{4}{=|}}
128 & 10 & \cellcolor{lightgray} & \cellcolor{lightgray} & \cellcolor{lightgray} & \cellcolor{lightgray} \\
& 15 & \cellcolor{

$Q=2^{ 20 }$, (14, 0.58752, 28, 990 KiB)
$Q=2^{ 40 }$, (-6, 0.58752, 28, 990 KiB)
$Q=2^{ 60 }$, (-26, 0.58752, 28, 990 KiB)
$Q=2^{ 80 }$, (-46, 0.58752, 28, 990 KiB)

Conjectured security calculations 1
$Q=2^{ 20 }$, (34, 0.85417, 28, 990 KiB)
$Q=2^{ 40 }$, (14, 0.85417, 28, 990 KiB)
$Q=2^{ 60 }$, (-6, 0.85417, 28, 990 KiB)
$Q=2^{ 80 }$, (-26, 0.85417, 28, 990 KiB)

Conjectured security calculations 2
$Q=2^{ 20 }$, (63, 0.87500, 28, 990 KiB)
$Q=2^{ 40 }$, (43, 0.87500, 28, 990 KiB)
$Q=2^{ 60 }$, (23, 0.87500, 28, 990 KiB)
$Q=2^{ 80 }$, (3, 0.87500, 28, 990 KiB)
